# Batch Optimization Examples

This notebook shows how to run multiple metric/optimizer jobs in a batch using `py_scripts/optimization_workflow.py`.
It also saves audio and plots with a consistent naming scheme matching `save_wav(add_info=True, add_time=True)`.

In [1]:
# Setup and target loading
import numpy as np
import pandas as pd
from IPython.display import display
from pathlib import Path

# Reuse the same configuration as test.ipynb
SR = 44100
DURATION = 2.0
FFT_PAD = 2
WAVEFORM_DTYPE = 'float32'  # 'float32' or 'float64'
FADE_IN_MS = 10.0
FADE_OUT_MS = 10.0

# Load target partials (example: cello)
target_path = 'tsv/voice-single2.tsv'
df = pd.read_csv(target_path, sep="\t")
display(df)
target_freqs = df["Frequency (Hz)"].to_numpy()
target_amps = (df["Amplitude"] / df["Amplitude"].max()).to_numpy()
# Extract a short name for naming (e.g., 'cello_single')
target_name = Path(target_path).stem


,Frequency (Hz),Amplitude
0,431.578947,0.000596
1,444.736842,0.003736
2,452.631579,0.006030
3,460.526316,0.111060
4,478.947368,0.002054
5,905.263158,0.000590
6,921.052632,0.016148
7,926.315789,0.012469
8,939.473684,0.001063
9,1386.842105,0.000786


## Define batch jobs
- `method`: one of `'de'` (Differential Evolution), `'da'` (Dual Annealing), `'bh'` (Basin Hopping).
- `metric`: any supported metric: `'pearson'`, `'mfcc'`, `'itakura_saito'`, `'spectral_convergence'`, `'cosine'`, `'euclidean'`, `'manhattan'`, `'kl'`.
- `kwargs`: optimizer-specific settings (e.g., `maxiter`, `workers`, `stepsize`).

In [2]:
from py_scripts.optimization_workflow import run_batch_jobs, BatchJob

jobs = [
    BatchJob(method='de', metric='pearson', kwargs={'maxiter': 500, 'workers': -1}),
    BatchJob(method='da', metric='pearson', kwargs={'maxiter': 500}),
    BatchJob(method='bh', metric='pearson', kwargs={'maxiter': 500, 'stepsize': 0.4}),
    
    # Add more combinations as needed, e.g.:
    BatchJob(method='de', metric='mfcc', kwargs={'maxiter': 500, 'workers': -1}),
    BatchJob(method='da', metric='mfcc', kwargs={'maxiter': 500}),
    BatchJob(method='bh', metric='mfcc', kwargs={'maxiter': 500, 'stepsize': 0.4}),

    BatchJob(method='de', metric='itakura_saito', kwargs={'maxiter': 500, 'workers': -1}),
    BatchJob(method='da', metric='itakura_saito', kwargs={'maxiter': 500}),
    BatchJob(method='bh', metric='itakura_saito', kwargs={'maxiter': 500, 'stepsize': 0.4}),

    BatchJob(method='de', metric='spectral_convergence', kwargs={'maxiter': 500, 'workers': -1}),
    BatchJob(method='da', metric='spectral_convergence', kwargs={'maxiter': 500}),
    BatchJob(method='bh', metric='spectral_convergence', kwargs={'maxiter': 500, 'stepsize': 0.4}),

    BatchJob(method='de', metric='cosine', kwargs={'maxiter': 500, 'workers': -1}),
    BatchJob(method='da', metric='cosine', kwargs={'maxiter': 500}),
    BatchJob(method='bh', metric='cosine', kwargs={'maxiter': 500, 'stepsize': 0.4}),

    BatchJob(method='de', metric='euclidean', kwargs={'maxiter': 500, 'workers': -1}),
    BatchJob(method='da', metric='euclidean', kwargs={'maxiter': 500}),
    BatchJob(method='bh', metric='euclidean', kwargs={'maxiter': 500, 'stepsize': 0.4}),   

    BatchJob(method='de', metric='manhattan', kwargs={'maxiter': 500, 'workers': -1}),
    BatchJob(method='da', metric='manhattan', kwargs={'maxiter': 500}),
    BatchJob(method='bh', metric='manhattan', kwargs={'maxiter': 500, 'stepsize': 0.4}),
    
    BatchJob(method='de', metric='kl', kwargs={'maxiter': 500, 'workers': -1}),
    BatchJob(method='da', metric='kl', kwargs={'maxiter': 500}),
    BatchJob(method='bh', metric='kl', kwargs={'maxiter': 500, 'stepsize': 0.4})   

]

rows = run_batch_jobs(
    target_freqs, target_amps, jobs,
    target_name=target_name,
    sr=SR, duration=DURATION, fft_pad=FFT_PAD, waveform_dtype=WAVEFORM_DTYPE,
    fade_in_ms=FADE_IN_MS, fade_out_ms=FADE_OUT_MS,
    seed=42,
)

# Summarize batch results (params/history omitted for brevity)
summary = pd.DataFrame([{k: v for k, v in r.items() if k not in ('params','history') } for r in rows])
display(summary.sort_values(['metric', 'best']))


FM 4-osc DE → Pearson:   0%|           0/500 [ETA: ?, Elapsed: 00:00]

KeyboardInterrupt: 

## Outputs and naming schema
Each job saves: 
- Audio (`.wav`) to `rendered_audio/optimized_output_fm_<target>_<method>_<metric>_<YYYYMMDD-HHMMSS>.wav`
- Plots (`.png`) to `rendered_plots/optimized_output_fm_<target>_<method>_<metric>_<YYYYMMDD-HHMMSS>_<type>.png`
  - `<type>` ∈ `time`, `spectrum`, `error`
- Final oscillator values to `tsv/final_values_fm_<target>_<method>_<metric>_<YYYYMMDD-HHMMSS>.tsv`

The naming follows the same `_method_metric_timestamp` pattern used by `save_wav(add_info=True, add_time=True)` for consistency across assets.

## Parallel processing note
- `workers` (parallel execution) is supported by Differential Evolution (`method='de'`).
- Dual Annealing (`'da'`) and Basin Hopping (`'bh'`) do not accept `workers`. Any unsupported kwargs are safely ignored by the batch runner.